In [ ]:
import numpy as np
import cv2 as cv
import glob
import PIL
from PIL import Image
from math import ceil, floor
from matplotlib import pyplot as plt
import os




In [ ]:
videos = []
videos.extend(glob.glob('*.mp4'))


In [ ]:
videocount = 0
for video in videos:
    videocount += 1
    vidcap = cv.VideoCapture(video)
    vidcap.set(cv.CAP_PROP_POS_MSEC,5000)      # just cue to 20 sec. position
    success,image = vidcap.read()
    if success:
        cv.imwrite(str(videocount) + '.jpg', image)     # save frame as JPEG file
    
    pass
    

In [ ]:
types = ('*.jpeg', '*.jpg','*.png') # the tuple of file types
images = []
for files in types:
    images.extend(glob.glob(files))

In [ ]:
frame_width = 1500
images_per_row = 4
padding = 4

             

img_width, img_height = 500,500
sf = (frame_width-(images_per_row-1)*padding)/(images_per_row*img_width)       #scaling factor
scaled_img_width = ceil(img_width*sf)                   #s
scaled_img_height = ceil(img_height*sf)

number_of_rows = ceil(len(images)/images_per_row)
frame_height = ceil(sf*img_height*number_of_rows) 

new_im = Image.new('RGB', (frame_width, frame_height))

i,j=0,0
for num, im in enumerate(images):
    if num%images_per_row==0:
        i=0
    im = Image.open(im)
    #Here I resize my opened image, so it is no bigger than 100,100
    im.thumbnail((scaled_img_width,scaled_img_height))
    #Iterate through a 4 by 4 grid with 100 spacing, to place my image
    y_cord = (j//images_per_row)*scaled_img_height
    new_im.paste(im, (i,y_cord))
    print(i, y_cord)
    i=(i+scaled_img_width)+padding
    j+=1

new_im.save("collage.jpg",optimize=True, progressive=True)

In [ ]:
def Histogram_Computation(Image):

    Image_Height = Image.shape[0]
    Image_Width = Image.shape[1]

    Histogram = np.zeros([256], np.int32)

    for x in range(0, Image_Height):
        for y in range(0, Image_Width):
            Histogram[Image[x,y]] +=1

    return Histogram



In [ ]:
Input_Image = cv.imread("collage.jpg",0) 

Histogram = Histogram_Computation(Input_Image)

In [ ]:
for i in range(0, Histogram.shape[0]):

    print( Histogram)

Plot_Histogram(Histogram)

In [ ]:
histlist = Histogram.tolist()


In [ ]:
numerical = list(range(256))

In [ ]:
import pandas as pd
import random
import numpy as np
import cv2 as cv
import glob
from math import ceil, floor
from matplotlib import pyplot as plt
import os




In [ ]:
lst1 = list(range(100))
lst2 =list(np.random.randint(low = 1,high=3000,size=100))
df = pd.DataFrame(
    {'No': lst1,
     'Bw': lst2,
    })

In [ ]:
df.to_csv('histogram.csv',index=False)

In [ ]:
histogram= df.to_dict('r')

In [ ]:
histogram

In [ ]:
from miditime.miditime import MIDITime
mymidi = MIDITime(300, 'collage.mid', 70, 3, 3)

In [ ]:
my_data_timed_b = [{'beat': mymidi.beat(d['No']), 'freq': d['Bw']} for d in histogram]

In [ ]:
start_timeb = my_data_timed_b[0]['beat']

In [ ]:
def mag_to_pitch_tunedb(freq):
    scale_pctb = mymidi.linear_scale_pct(min([x['Bw'] for x in histogram]), 3000, freq)


    c_major = ["C", "C#", "Db", "D", "D#", "Eb", "E", "F", "F#", "Gb", "G", "G#", "Ab", "A", "A#", "Bb", "B"]


    noteb = mymidi.scale_to_note_classic(scale_pctb, c_major)



    midi_pitchb = mymidi.note_to_midi_pitch(noteb)

    
    return midi_pitchb



In [ ]:
note_listb = []

for d in my_data_timed_b:
    note_listb.append([
        d['beat'] - start_timeb,
        mag_to_pitch_tunedb(d['freq']),
        100,  # velocity
        1  # duration, in beats
    ])


In [ ]:
# Add a track with those notes
mymidi.add_track(note_listb)
# Output the .mid file
mymidi.save_midi()

In [ ]:
from pydub import AudioSegment
from pydub.generators import Sine
from collections import defaultdict
from mido import MidiFile

In [ ]:
def note_to_freq(note, concert_A=440.0):
    
    return (2.0 ** ((note - 69) / 12.0)) * concert_A

mid = MidiFile("collage.mid")
output = AudioSegment.silent(mid.length * 2000.0)

tempo = 150 # bpm
output = output.low_pass_filter(1000)

def ticks_to_ms(ticks):
    tick_ms = (60000.0 / tempo) / mid.ticks_per_beat
    return ticks * tick_ms
  

for track in mid.tracks:
  
    current_pos = 0.0

    current_notes = defaultdict(dict)

    for msg in track:
        current_pos += ticks_to_ms(msg.time)

        if msg.type == 'note_on':
            current_notes[msg.channel][msg.note] = (current_pos, msg)

        if msg.type == 'note_off':
            start_pos, start_msg = current_notes[msg.channel].pop(msg.note)

            duration = current_pos - start_pos

            signal_generator = Sine(note_to_freq(msg.note))
            rendered = signal_generator.to_audio_segment(duration=duration, volume=-20).fade_out(300).fade_in(30)

            output = output.overlay(rendered, start_pos)

output.export("generatedmemory.wav", format="wav")

In [ ]:
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt
import os
from mobilechelonian import Turtle

In [ ]:
name ='generatedmemory'
file = name+".wav"
title =name+".ps"
png = name + '.png'
frame_count=0


In [ ]:
signal, sample_rate = librosa.load(file)

In [ ]:
FRAME_SIZE = 2028
HOP_LENGTH = 250000


In [ ]:
rms = librosa.feature.rms(signal, frame_length=FRAME_SIZE, hop_length=HOP_LENGTH)[0]
frames = range(len(rms))
t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH)

In [ ]:
spec = []
for i in rms:
    spec.append(i*120)

In [ ]:
len(rms)

In [ ]:
import turtle
import time
from turtle import Screen, Turtle
from random import randint
import random
import PIL.Image
from tkinter import *
import itertools
import io

In [ ]:
# audio/visual animation & final output of sound to image
current_directory = os.getcwd()
image_path = os.path.join(current_directory, r'new_folder')
if not os.path.exists(image_path):
    os.makedirs(image_path)
    
window = turtle.Screen()
window.title(str(name))
window.setup(width = 650, height = 650)
cv = turtle.getcanvas()

turtle.hideturtle()
turtle.speed(0)
turtle.width(2)
turtle.bgcolor('white')
time.sleep(5)
warna = ['#f9a931','#2568b6','#fb4a16','#026768','#c4a4cd','#3c3f45'
         ]

bgc = ['#43bcd9', '#fec009','#fc8a17','#9a9b9f','#506d5b','#e1907f','#dbcec5','#f2bdcd']
w= random.choice(bgc)
turtle.penup()
turtle.pencolor(w)
turtle.width(3)
turtle.goto(-325,325)
turtle.pendown()
turtle.fillcolor(w)
turtle.begin_fill()
turtle.goto(0,325)
turtle.goto(0,0)
turtle.goto(-325,0)
turtle.goto(-325,325)
turtle.end_fill()
turtle.penup()



w= random.choice(bgc)
turtle.penup()
turtle.pencolor(w)
turtle.width(3)
turtle.goto(0,325)
turtle.pendown()
turtle.fillcolor(w)
turtle.begin_fill()
turtle.goto(325,325)
turtle.goto(325,0)
turtle.goto(0,0)
turtle.goto(0,325)
turtle.end_fill()
turtle.penup()


turtle.penup()
turtle.pencolor(w)
turtle.width(3)
turtle.setheading(0)
turtle.goto(-325,0)
turtle.pendown()
turtle.fillcolor(w)
turtle.begin_fill()
turtle.goto(0,0)
turtle.goto(0,-325)
turtle.goto(-325,-325)
turtle.goto(-325,0)
turtle.end_fill()
turtle.penup()



w= random.choice(bgc)
turtle.penup()
turtle.pencolor(w)
turtle.width(3)
turtle.setheading(0)
turtle.goto(0,0)
turtle.pendown()
turtle.fillcolor(w)
turtle.begin_fill()
turtle.goto(325,0)
turtle.goto(325,-325)
turtle.goto(0,-325)
turtle.goto(0,0)
turtle.end_fill()
turtle.penup()





density= [0,2,5]

sudut = [0,90,180,360]
for s in spec:
    for i in range(0,1):
        d= random.choice(density)
        w= random.choice(warna)
        turtle.pencolor(w)
        turtle.goto(randint(-320,320),randint(-320,320))
        turtle.width(d)
        turtle.setheading(random.choice(sudut))
        turtle.pendown()
        turtle.circle(s*6.5,180)
        turtle.penup()
  

    
 
    turtle.pencolor('black')
    turtle.goto(randint(-300,300),randint(-300,300))
    turtle.width(2)
    turtle.fillcolor('black')
    turtle.begin_fill()
    turtle.pendown()
    turtle.circle(s*5)
    turtle.end_fill()
    turtle.penup()
    turtle.goto(turtle.position()[0]-5, turtle.position()[1]-1)
    w = random.choice(warna)
    turtle.pencolor(w)
    turtle.fillcolor(w)
    turtle.begin_fill()
    turtle.pendown()
    turtle.circle(s*5)
    turtle.end_fill()
    turtle.penup()
    
    
    turtle.pencolor('black')
    turtle.goto(randint(-300,300),randint(-300,300))
    turtle.width(2)
    turtle.fillcolor('black')
    turtle.begin_fill()
    turtle.pendown()
    turtle.circle(s*7.5)
    turtle.end_fill()
    turtle.penup()
    turtle.goto(turtle.position()[0]-5, turtle.position()[1]-1)
    w = random.choice(warna)
    turtle.pencolor(w)
    turtle.fillcolor(w)
    turtle.begin_fill()
    turtle.pendown()
    turtle.circle(s*7.5)
    turtle.end_fill()
    turtle.penup()
    



    
    d=random.choice(density)
    turtle.width(d)
    for i in range(0,6):
        turtle.goto(randint(-320,320),randint(-320,320))
        turtle.pendown()
        turtle.fillcolor(w)
        turtle.begin_fill()
        turtle.forward(i*20)
        turtle.right(90)
        turtle.end_fill()
        turtle.penup()

turtle.penup()
turtle.goto(5,-300)
turtle.pendown()
turtle.pencolor('black')
turtle.pendown()
turtle.write('monalisa' ,move=False,align="center", font=("Arial", 20)) # monalisa can be passed to text input



        


       




        

cv.postscript(file=title, colormode='color')

img = PIL.Image.open(title)
img.load(scale=10)    
img.save(png,'png')  #final result from sound to image


turtle.done()
print('is done')


